In [1]:
import folium
import branca
import numpy as np
import pandas as pd
from MoviaBusDataset import MoviaBusDataset
import matplotlib.pyplot as plt

In [2]:
train = MoviaBusDataset('../data/train', 
                        interpolation=True)

In [169]:
df = pd.concat(train.dataframes)

df_jerk = df.diff().abs().mean().to_frame()

df_jerk['wayId'], df_jerk['source'], df_jerk['target'] = df_jerk.index.str.split(':').str
df_jerk = df_jerk.reset_index().rename(columns={0:'mean_jerk'})


In [170]:
#Create two colormaps, one for the mean speed and one for the number of meassurements
cm_mean = branca.colormap.linear.YlOrRd_04.scale(df_jerk['mean_jerk'].min(),df_jerk['mean_jerk'].max())

#Create a map object in valby
m = folium.Map(location=[55.66,12.515], zoom_start=15)

#Define style function, that change the color depending on the value of the mean speed
def style_trafic_mean(feature):
    wayid = str(feature['properties']['WayId'])
    
    
    val = df_jerk[df_jerk['wayId']==wayid]['mean_jerk'].values
    
   
    
    if np.size(val) >= 1:
        val = val.mean()
    else:
        val = 0
    if val < 1:
         return {
                    'fillOpacity': 0,
                    'weight': 0,
                    'color':0
            }
    return {
            'fillOpacity': 1,
            'weight': 10,
            'color':cm_mean(val)
            }

    
        
#Add the geojson with colors
g=folium.GeoJson('../data/road_network.geojson',name='mean speed', style_function=style_trafic_mean)
folium.GeoJsonTooltip(['WayId']).add_to(g)
g.add_to(m)
#This is a hack to show a colorbar, and also include all the streets
m.choropleth(geo_data='../data/road_network.geojson'
             , data=df_jerk, columns=['wayId','mean_jerk']
             , key_on='feature.properties.WayId'
             , fill_color='YlOrRd'
             , name='Streets'
             , legend_name='Mean acceleration [m/s^2]')
#Add a control layer to our map
folium.LayerControl().add_to(m)

#Show the resulting map
m